In [1]:
import matplotlib.pyplot as plt
import os
import numpy as np
import pathlib
from scipy.io import loadmat
import pandas as pd
import pywt
from scipy.signal import find_peaks,peak_widths
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
#cut all the QRS wave and use to classify


In [ ]:
data = 'New_more'
data_dir = pathlib.Path(data)
data_dir

In [ ]:
data_dict = {
    'Normal': list(data_dir.glob('Normal/*')),
    'AF': list(data_dir.glob('AF/*'))
}
    

In [ ]:
data_lavel_dict = {
    'Normal': 0,
    'AF': 1
}
    

In [ ]:
data_dict

In [ ]:
dataset = []
y = []
for lavel, datas in data_dict.items():
    for sig in datas:
        data = np.load(str(sig))
        Data=data
        dataset.append(Data)
        y.append(data_lavel_dict[lavel])

In [ ]:
dataset = np.array(dataset)

In [ ]:
len(dataset)

In [ ]:
y

In [ ]:
def feature_extractor(sample_frequency,x):
    #sample_frequency = 360
    fs = sample_frequency
    t = np.arange(0,len(x))
    tx = t/fs

    #signal filtration and possible Peak detection
    coeffs1 = pywt.wavedec(x,'sym4',level = 4,mode= 'periodic')
    z0 = np.zeros(len(coeffs1[0]))
    z1 = np.zeros(len(coeffs1[2]))
    z3 = np.zeros(len(coeffs1[3]))
    z4 = np.zeros(len(coeffs1[4]))

    coeffs1[3] = z3
    coeffs1[4] = z4

    y1 = pywt.waverec(coeffs1,'sym4',mode = 'periodic')

    avg1 = np.mean(abs(y1))

    peaks1 = find_peaks(y1,height=avg1,distance = 50)

    nop1 = peaks1[0]
    posi1 = peaks1[1]
    posi1 = posi1['peak_heights']

    #applying wavelet transform
    coeffs = pywt.wavedec(y1,'sym4',level = 4,mode = 'periodic')

    #Removing some part of coefficient for proper R peak detection purpose
    z1 = np.zeros(len(coeffs[0]))
    z3 = np.zeros(len(coeffs[3]))
    z4 = np.zeros(len(coeffs[4]))
    coeffs[0] = z1
    coeffs[3] = z3
    coeffs[4] = z4

    #reconstracting the signal
    y = pywt.waverec(coeffs,'sym4',mode = 'periodic')

    #finding the R_peaks from the reconstracted signal
    avg = np.mean(abs(y))
    peaks = find_peaks(y,height = avg*3,distance = 200)

    nop = peaks[0]
    posi = peaks[1]
    posi = posi['peak_heights']
    number_Rpeak = len(nop)

    #Finding Number of beats per minute or Heartbeat
    nump = len(nop)
    time = len(x)/fs
    nob = (nump*60)/time

    #Original R peak and height
    new_nop = []
    for i in nop1:
        for j in nop:
            if abs(i-j)< 3:
                new_nop.append(i)
                continue
            else:
                pass

    r_real_height = []
    for i in new_nop:
        r_real_height.append(y1[i])

    #finding height of R-peaks
    avg_height = round(np.mean(r_real_height),2)
    minimum_height = round(min(r_real_height),2)
    maximum_height = round(max(r_real_height),2)

    #finding peak width
    results_half = peak_widths(y1, new_nop, rel_height=0.5)
    width = results_half[0]

    avrg_width = round(np.mean(width),2)
    maximum_width = round(max(width),2)
    minimum_width = round(min(width),2)

    #finding R-R peak Distance
    new_nop = np.array(new_nop)
    nop_s = new_nop/fs
    peak_dis = []

    for i in range(len(nop_s)-1):
        dis = abs(nop_s[i]-nop_s[i+1])
        dis = round(dis,1)
        peak_dis.append(dis)

    avr_distance = round(np.mean(peak_dis),2)
    max_distance = round(max(peak_dis),2)
    min_distance = round(min(peak_dis),2)

    #finding discontinuty
    disc = []
    for i in range(len(peak_dis)):
        if 0.6 <= peak_dis[i] <= 1.2:
            disc.append(0)
        else:
            disc.append(1)

    if max(disc) == 1:
        discontinuty = 1
    else:
        discontinuty = 0

    #P-peaks detection
    peaks_for_p = find_peaks(y1,height= 0*avg1,distance = 20)
    nop_for_p = peaks_for_p[0]
    posi_for_p = peaks_for_p[1]
    posi_for_p = posi_for_p['peak_heights']

    p_peaks = []
    p_height = []
    p_R_dis = []
    nopp = new_nop

    #def p_detection(p_value):

    for i,n in enumerate(nopp):
        #print('n = ',n)
        #print('i = ',i)
        for j,m in enumerate(nop_for_p):
            #print('m = ',m)

            if m == n:
                #print('found equal')

                if (j-1) < 0:
                    k=j
                elif (j-1) == 0:
                    k = j-1
                else:
                    k = j-1
                #print('not zero')
                l = nop_for_p[k]
                #print('l = ',l)


                if (i-1) < 0:
                    p=i
                elif (i-1) == 0:
                    p = i-1
                else:
                    p = i-1
                #print('p = ',p)

                for qw in range(10):
                    if l != nopp[p]:
                        #print(n-l)

                        if (fs*0.12)-5 < n-l < (fs*0.20)+5:
                            p_peaks.append(l)
                            p_height.append(posi_for_p[k])
                            p_R_dis.append(abs(n-l))
                            break
                        else:
                            k = k-1
                            l = nop_for_p[k]

                    else:
                        pass


            else:
                pass

    #finding Height of p_peaks
    number_ppeaks = len(p_peaks)
    max_height = round(max(p_height),2)
    min_height = round(min(p_height),2)
    avg_pheight = round(np.mean(p_height),2)

    #P-R Interval or Distance
    p_R_dis = np.array(p_R_dis)/fs
    max_PR_dis = round(max(p_R_dis),2)
    min_PR_dist = round(min(p_R_dis),2)
    avg_PR_dis = round(np.mean(p_R_dis),2)

    #QRS interval detection
    r_y = -1*y1
    avg2 = np.mean(abs(r_y))
    QRS_peaks = find_peaks(r_y,height = -1*avg2*2,distance = 5)
    QRS_nop1 = QRS_peaks[0]
    QRS_posi = QRS_peaks[1]
    QRS_posi = QRS_posi['peak_heights']

    QRS_nop = np.append (QRS_nop1, new_nop)
    QRS_nop.sort()

    #QRS interval detection
    Q_peaks = []
    Q_hei = []
    Q_dis = []
    s_peaks = []
    s_hei = []
    s_dis = []
    nopp = new_nop

    for i,n in enumerate(nopp):
        #print('n = ',n)
        #print('i = ',i)
        for j,m in enumerate(QRS_nop):
            #print('m = ',m)

            if m == n:
                #print('found equal')

                if (j-1) < 0:
                    k=j
                elif (j-1) == 0:
                    k = j-1

                else:
                    k = j-1
                #for s peak
                if j+1 > len(QRS_nop):
                    e = j
                elif j+1 == len(QRS_nop):
                    e = j
                else:
                    e = j+1

                #print('not zero')
                l = QRS_nop[k]
                #print('l = ',l)
                s = QRS_nop[e]


                if (i-1) < 0:
                    p=i
                elif (i-1) == 0:
                    p = i-1
                else:
                    p = i-1

                #for s peak
                if i+1 > len(nopp):
                    g = i
                elif i+1 == len(nopp):
                    g = i
                else:
                    g = i+1
                #print('p = ',p)  
                if l != nopp[p]:
                    #print(n-l)

                    if n-l < 25:
                        #print("p R =",l,m)
                        Q_peaks.append(l)
                        Q_hei.append(y1[l])

                    else:
                        Q_peaks.append(n-8)
                        Q_hei.append(0)
                else:
                    Q_peaks.append(n-8)
                    Q_hei.append(0)

                if s != nopp[g]:

                    if abs(n-s) < 25:
                        #print("p R =",l,m)
                        s_peaks.append(s)
                        s_hei.append(y1[s])


                    else:
                        s_peaks.append(n+12)
                        s_hei.append(0)
                else:
                    s_peaks.append(n+12)
                    s_hei.append(0)
            else:
                pass

    q_peaks1 = np.array(Q_peaks)
    S_peaks1 =np.array(s_peaks)
    QRS_intervel = abs(S_peaks1 - q_peaks1 )

    #finding Height of p_peaks
    '''
    s_hei
    Q_hei
    '''
    avg_hei_s = round(np.mean(s_hei))
    avg_hei_q = round(np.mean(Q_hei))

    #QRS Interval or Distance
    QRS_intervel = (QRS_intervel)/fs
    max_QRS_dis = round(max(QRS_intervel),2)
    min_QRS_dist = round(min(QRS_intervel),2)
    avg_QRS_dis = round(np.mean(QRS_intervel),2)

    #feature list
    feature = [nob,maximum_height,minimum_height,avg_height,maximum_width,minimum_width,avrg_width,max_distance,min_distance,avr_distance,discontinuty,number_Rpeak,number_ppeaks,max_height,min_height,avg_pheight,max_PR_dis,min_PR_dist,avg_PR_dis,max_QRS_dis,min_QRS_dist,avg_QRS_dis,abs(avg_hei_s),abs(avg_hei_q)]
    return(feature)
    #feature = [nob,maximum_height,minimum_height,avg_height,maximum_width,minimum_width,avrg_width,max_distance,min_distance,avr_distance,discontinuty,max_height,min_height,avg_pheight,max_PR_dis,min_PR_dist,avg_PR_dis,max_QRS_dis,min_QRS_dist,avg_QRS_dis]
    #return(feature)

In [ ]:
plt.plot(dataset[653])
#plt.xlim(0,1000)

In [ ]:
len(dataset)

In [ ]:
data_set1 = dataset
y1 = y
x = []
for i in range(len(dataset)):
    f = feature_extractor(300,dataset[i])
    #f = f[1:11]
    #al = np.array(al)
    #f.append(al)
    #f = np.array(f)
    x.append(f)

In [ ]:
len(x)

In [ ]:
len(y)

In [ ]:
x[10]

In [ ]:
len(x[5])

In [ ]:
len(x)

In [ ]:
x = np.array(x)
y = np.array(y)

In [ ]:
x.shape

# for detecting highest value:

In [ ]:
#flat = x.flatten()

In [ ]:
#flat.shape

In [ ]:
y.shape

In [ ]:
#x = x.reshape(1542,11*30)

In [ ]:
x.shape

In [ ]:
#from sklearn.preprocessing import MinMaxScaler
#mm = MinMaxScaler().fit(x)

In [ ]:
from sklearn.preprocessing import StandardScaler
stnd = StandardScaler().fit(x)

In [ ]:
x = stnd.transform(x)

In [ ]:
x

In [ ]:
y

In [ ]:
from keras.utils import to_categorical
#y = to_cateorical(y)
#g = to_categorical(y)
#y = g
#y

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state=0)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
num_classes = 2
model1 = Sequential([
    #layers.Normalization(axis=None),
    layers.Dense(24,activation = 'relu'),
    layers.Dense(15,activation = 'relu'),
    layers.Dense(10,activation = 'relu'),
    layers.Dense(10,activation = 'relu'),
    layers.Dense(num_classes, activation = 'sigmoid')
    
])

model1.compile(optimizer = 'adam',
             loss=tf.keras.losses.SparseCategoricalCrossentropy(
                from_logits=True
            ),
             metrics = ['accuracy'])


In [ ]:
history = model1.fit(x_train, y_train, epochs = 100)

In [ ]:
model1.evaluate(x_test,y_test)

In [ ]:
predicted = model1.predict(x_test)

In [ ]:
predicted1 = np.round(predicted)

In [ ]:

predicted2 = []
for i in range(len(predicted1)):
    z = np.argmax(predicted1[i])
    predicted2.append(z)
   

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report

In [ ]:
print(classification_report(y_test,predicted2))

In [ ]:
import seaborn as sns
cm = confusion_matrix(y_test,predicted2)
sns.heatmap(cm,annot = True)
plt.show()


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])

In [ ]:
model1.save('Model_big_data.h5')

In [ ]:
x.shape

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

# Shuffeling dataset

In [ ]:
from sklearn.utils import shuffle

In [ ]:
x2,y = shuffle(x,y)

In [ ]:
x2[10]

In [ ]:
print(y)

In [ ]:
def buid_classifier():
    num_classes = 2
    model1 = Sequential([
        layers.Flatten(),
        #layers.Normalization(axis=None),
        layers.Dense(24,activation = 'relu'),
        layers.Dense(15,activation = 'relu'),
        layers.Dense(10,activation = 'relu'),
        layers.Dense(10,activation = 'relu'),
        layers.Dense(num_classes, activation = 'sigmoid')

    ])

    model1.compile(optimizer = 'adam',
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(
                    from_logits=True
                ),
                 metrics = ['accuracy'])
    return model1


In [ ]:
classifiers = KerasClassifier(build_fn= buid_classifier, epochs = 100)
accuracies = cross_val_score(estimator= classifiers, X= x2, y = y, cv = 9)

In [ ]:
accuracies

In [ ]:
means = accuracies.mean()

In [ ]:
means

In [ ]:
predicted = model.predict(x_test)

In [ ]:
predicted1 = np.round(predicted)

In [ ]:
predicted2 = []
for i in range(len(predicted1)):
    z = np.argmax(predicted1[i])
    predicted2.append(z)

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report

In [ ]:
print(classification_report(y_test,predicted2))

In [ ]:
import seaborn as sns
cm = confusion_matrix(y_test,predicted2)
sns.heatmap(cm,annot = True)
plt.show()